In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

In [ ]:
data_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[.5], std=[.5])
])

In [ ]:
batch_size = 64

trainset = torchvision.datasets.FashionMNIST(root='./data', train=True,
                                        download=True, transform=data_transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=0)

testset = torchvision.datasets.FashionMNIST(root='./data', train=False,
                                       download=True, transform=data_transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=0)


In [ ]:
class CNN(nn.Module):
    def __init__(self, activation_func):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 64, 3, 1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(64, 32, 3, 1)
        self.fc1 = nn.Linear(32 * 5 * 5, 256)
        self.fc2 = nn.Linear(256, 10)
        self.activation = activation_func

    def forward(self, x):
        x = self.conv1(x)
        x = self.activation(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = self.activation(x)
        x = self.pool(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = self.activation(x)
        x = self.fc2(x)
        return x


In [ ]:
model = CNN(activation_func=nn.ELU())
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
num_epochs = 1
for epoch in range(0, num_epochs, 1):
  model.train()
  all_labels = []
  all_preds = []
  running_loss = 0.0

  for inputs, targets in trainloader:
    optimizer.zero_grad()
    outputs = model(inputs)
    targest = targets.squeeze().long()
    loss = loss_func(outputs, targets)
    loss.backward()
    optimizer.step()
    running_loss = running_loss + loss.item()
    _, preds = torch.max(outputs, 1)

    all_preds.extend(preds.cpu().numpy())
    all_labels.extend(targets.cpu().numpy())

  train_loss = running_loss / len(trainloader)
  print(
        f"Epoch {epoch + 1}/{num_epochs}, "

    )

Epoch 1/1, 


In [ ]:
model.eval()
all_preds = []
all_labels = []
all_probas = []
with torch.no_grad():
    for inputs, targets in testloader:
        outputs = model(inputs)
        targets = targets.squeeze().long()
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(targets.cpu().numpy())
test_preds = np.array(all_preds)
test_labels = np.array(all_labels)
f1 = f1_score(test_labels, test_preds, average='macro')

In [ ]:
print(f"F1 Score: {f1:.4f}")

F1 Score: 0.8803


In [ ]:
totalparam = 0
for param in model.parameters():
  totalparam = totalparam + param.numel()

In [ ]:
print(totalparam)

226730
